<a href="https://colab.research.google.com/github/SivananthamP/My_Projects/blob/main/my_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
from diffusers import StableDiffusionPipeline, StableDiffusionInpaintPipeline, DPMSolverMultistepScheduler
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

SYSTEM_PROMPT = """
You are a professional Stable Diffusion prompt engineer.
Convert the user text into a detailed cinematic realistic image prompt.
Add lighting, camera, environment, style, and realism details.
expressive eyes, soft smile, cinematic lighting, warm tones, bokeh background,
DSLR photography, ultra realistic, high detail, 8k, sharp focus, masterpiece,
dramatic lighting, photorealistic skin texture.

Return ONLY the prompt. Do NOT explain.
"""

def enhance_prompt(user_text):
    prompt = SYSTEM_PROMPT + "\nUser: " + user_text + "\nPrompt:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.0,
        do_sample=False,
        num_beams=5
    )

    enhanced = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return enhanced.split("Prompt:")[-1].strip()

# ================= LOAD SD PIPELINES =================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Text-to-image
pipe_txt2img = StableDiffusionPipeline.from_pretrained(
    "SG161222/Realistic_Vision_V5.1_noVAE",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32
).to(device)

# Inpainting
pipe_inpaint = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting"
).to(device)
pipe_inpaint.scheduler = DPMSolverMultistepScheduler.from_config(pipe_inpaint.scheduler.config)

# ================= NEGATIVE PROMPT =================
negative_prompt = (
    "low quality, worst quality, blurry, pixelated, noise, grainy, jpeg artifacts, "
    "bad anatomy, bad hands, extra fingers, missing fingers, deformed hands, "
    "bad face, ugly, distorted face, cross-eye, extra limbs, mutated, "
    "text, watermark, logo, cropped, out of frame, duplicate, "
    "overexposed, underexposed, low resolution, poorly drawn"
)

# ================= GENERATE IMAGE =================
def generate_image(prompt):
    enhanced = enhance_prompt(prompt)
    print("Enhanced Prompt:", enhanced)

    image = pipe_txt2img(
        prompt=enhanced,
        negative_prompt=negative_prompt,
        guidance_scale=10
    ).images[0]

    image.save("output.png")
    return image, enhanced

# ================= INPAINT IMAGE =================
def inpaint_image(prompt, input_image):
    enhanced = enhance_prompt(prompt)
    print("Enhanced Prompt:", enhanced)

    image = input_image.resize((768,768))

    # create center mask
    mask = Image.new("L", image.size, 0)
    mask_size = 300
    start = (image.size[0] - mask_size) // 2
    end = start + mask_size
    mask.paste(255, (start, start, end, end))

    result = pipe_inpaint(
        prompt=enhanced,
        negative_prompt=negative_prompt,
        image=image,
        mask_image=mask,
        guidance_scale=9
    )

    final_image = result.images[0]
    final_image.save("inpainted_result.png")
    return final_image, enhanced

# ================= GRADIO UI =================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎨 SMART AI Image GENERATOR")

    with gr.Tab("🖼️ Text to Image"):
        txt_prompt = gr.Textbox(label="Enter Prompt", placeholder="a man proposing to a girl with a rose")
        btn_generate = gr.Button("Generate Image 🚀")
        out_img = gr.Image()
        out_prompt = gr.Textbox(label="Enhanced Prompt")

        btn_generate.click(generate_image, inputs=txt_prompt, outputs=[out_img, out_prompt])

    with gr.Tab("🧠 Inpainting"):
        in_prompt = gr.Textbox(label="Edit Prompt")
        upload_img = gr.Image(label="Upload Image")
        btn_inpaint = gr.Button("Inpaint ✨")
        out_inpaint = gr.Image()
        out_prompt2 = gr.Textbox(label="Enhanced Prompt")

        btn_inpaint.click(inpaint_image, inputs=[in_prompt, upload_img], outputs=[out_inpaint, out_prompt2])

demo.launch(share=True)



Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/396 [00:00<?, ?it/s]

StableDiffusionSafetyChecker LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/safety_checker
Key                                               | Status     |  | 
--------------------------------------------------+------------+--+-
vision_model.vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/tmp/ipython-input-1717990833.py:109: DeprecationWarning: The 'theme' paramete

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2bcc0c33155bea2a51.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
